<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/DL/Group_3DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning with PyTorch
## 3D Convolutional Neural Network on Group Brain fMRI
Contributors: Stacey Rivet Beck, Ben Merrill

- Implement 3DCNN from paper: REALLY GREAT PAPERS
  - Nguyen et al. http://proceedings.mlr.press/v136/nguyen20a/nguyen20a.pdf
  - Wang et al. https://arxiv.org/pdf/1801.09858.pdf (discusses more in detail the input data shapes and processing)
  
  - Inputs: 84@ x * y * z ; one fmri time series at a time, not concatenated
  - Basic Architecture:
        First layer is generating temporal descriptors of the voxels
        Conv1 1 x 1 x 1 filter, output = 32, stride = 1, ReLU, BatchNorm
        Conv2 7 x 7 x 7 filter, output = 64, stride = 2, ReLU, BatchNorm
        Conv3 3 x 3 x 3 fitler, output = 64, stride = 2, ReLU, BatchNorm
        Conv4 3 x 3 x 3 fitler, output = 128, stride = 2, ReLU, BatchNorm
        Global Average Pooling on final feature maps ->
        Flattened maps size 128?
        Fully connected layer size 64
        Fully connected layer size 2 (2 way classification, one for each class) -> softmax

        Optimized with Adam, standard parameters (β1=0.9 and β2=0.999)
        Batched at 32, but we may need to batch smaller due to GPU compute
        Learning Rate = 0.001, gradual decay after Val loss plateaued after 15 epochs
        Cross entropy Loss
        Employ early stopping
        In Wang et al. they used data for visualization, same size as input data, though are reduced in time dimension to be mapped on fsaverage surface. 
        
  

## Importing Dataset and Labels

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git

# Change directory into cloned repo DL folder
%cd teambrainiac/source/DL

# !ls

fatal: destination path 'teambrainiac' already exists and is not an empty directory.
/content/teambrainiac/source/DL


### Load path_config.py

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config (1).py
User uploaded file "path_config.py" with length 196 bytes


## Import Packages

In [4]:
# Possible Missing Packages
!pip install boto3
!pip install nilearn

In [5]:
# General Library Imports
import re
import scipy.io
import os
import pickle
import numpy as np
import nibabel as nib
import pandas as pd
import boto3
import tempfile
import tqdm
import random
from path_config import mat_path
from botocore.exceptions import ClientError
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

# From Local Directory
from access_data_dl import *
from process_dl import *

# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import TensorDataset

#import torchvision.transforms as transforms
from torch.nn import ReLU, CrossEntropyLoss, Conv3d, Module, Softmax, AdaptiveAvgPool3d
from torch.optim import Adam, SGD

#from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader


## Import Group fMRI Data from AWS

In [6]:
%%time
path = 'dl/partition_train_4.pkl'
train_images = access_load_data(path, False)

CPU times: user 32.7 s, sys: 53.7 s, total: 1min 26s
Wall time: 1min 30s


In [7]:
train_images['images'].shape

torch.Size([1512, 1, 79, 95, 79])

## Build Test Batch
Use this code only if you are changing the model based on a subset of the data

In [8]:
# train_batch_size = 18

# x = train_images['images'][:train_batch_size]
# y = train_images['labels'][:train_batch_size]

## Build Dataloader

In [9]:
# May move this into the training bit
# bs = 150

# x_train = train_images['images']
# y_train = train_images['labels']

# ds = TensorDataset(x_train, y_train)
# dl = DataLoader(ds, batch_size = bs, shuffle=True)

## Build Model

In [10]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    #Conv1
    self.conv1 = nn.Conv3d(in_channels = 1, 
                           out_channels = 32, 
                           kernel_size = (1,1,1), 
                           stride = (1,1,1)
                           )
    self.bn1 = nn.BatchNorm3d(32)
    self.conv2 = nn.Conv3d(in_channels = 32, 
                           out_channels = 64, 
                           kernel_size = (7,7,7),
                           stride = (2,2,2)
                           )
    self.bn2 = nn.BatchNorm3d(64)
    self.conv3 = nn.Conv3d(in_channels = 64, 
                           out_channels = 64, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn3 = nn.BatchNorm3d(64)
    self.conv4 = nn.Conv3d(in_channels = 64, 
                           out_channels = 128, 
                           kernel_size = (3,3,3),
                           stride = (2,2,2)
                           )
    self.bn4 = nn.BatchNorm3d(128) 
    self.pool1 = nn.AdaptiveAvgPool3d((1,1,1)) #Global Average Pool, takes the average over last two dimensions to flatten 
  
                                                             
    # Fully connected layer
    self.fc1 = nn.Linear(128,64) # need to find out the size where AdaptiveAvgPool 
    self.fc2 = nn.Linear(64, 2) # left with 2 for the two classes                     

  def forward(self, xb):
    xb = self.bn1(F.relu(self.conv1((xb))))
    xb = self.bn2(F.relu(self.conv2((xb)))) # Takes a long time
    xb = self.bn3(F.relu(self.conv3((xb))))
    xb = self.bn4(F.relu(self.conv4((xb))))
    xb = self.pool1(xb)
    xb = xb.view(xb.shape[:2])
    xb = self.fc1(xb)
    xb = self.fc2(xb)
    return xb      

    

In [11]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
# print("First model training on GPU")
# print(model)

# Initialize other parameters
epochs = 10
learning_rate = 0.001
loss_func = F.cross_entropy
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

In [12]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()



# Use to load GPU model
# model.load_state_dict(torch.load(path))



In [13]:

def run_cnn(model, epochs, learning_rate, loss_func, opt, dl):
  metrics_dict = {}
  # Run Model
  for epoch in range(1, 1+epochs):
    i = 1
    accuracy_list = []
    loss_list = []
    model.train()
    print('epoch', epoch)
    for xb, yb in dl:
      print('batch', i)
      i += 1

      xb = xb.float()
      pred = model(xb)
      loss_batch = loss_func(pred, yb)
      loss_list.append(loss_batch)
      accuracy_batch = accuracy(pred, yb)
      if int(accuracy_batch) == 1:
        print('Perfect Accuracy\nStopping early to avoid overfitting\n\n')
        break


      accuracy_list.append(accuracy_batch)

      print('Batch Loss', loss_batch)
      print('Batch Accuracy', accuracy_batch)

      loss_batch.backward()
      opt.step()
      opt.zero_grad()

    model.eval()
    
    print('Saving model')
    model_name = 'cnn_fmri_initial_model.pt'
    path = F"/content/gdrive/My Drive/{model_name}" 
    torch.save(model.state_dict(), path)

    metrics_dict['epoch_'+str(epoch)] = {'accuracy':accuracy_list, 'loss':loss_list}

    print('epoch', epoch, 'finished\n')
    try:
      past_epoch_accuracies = [sum(metrics_dict['epoch_'+str(epoch-2)]['accuracy']), sum(metrics_dict['epoch_'+str(epoch-1)]['accuracy'])]
      current_epoch_accuracy = sum(metrics_dict['epoch_'+str(epoch)]['accuracy'])
      if past_epoch_accuracies[0] > current_epoch_accuracy and past_epoch_accuracies[1] > current_epoch_accuracy:
        print('Early stop to avoid overfitting\nModel accuracies did not decrease for two epochs')
        return model, metrics_dict

    except:
      pass
  
  return model, metrics_dict
  


In [14]:
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
# print("First model training on GPU")
# print(model)

# Initialize other parameters
epochs = 10
learning_rate = 0.001
loss_func = F.cross_entropy
opt = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum



In [15]:
load_recent_model = True

if load_recent_model:
  path = '/content/gdrive/My Drive/cnn_fmri_model_after_train_3.pt'
  model.load_state_dict(torch.load(path))

In [16]:

metrics_dict = {}
for i,image_index in enumerate(range(0, train_images['images'].shape[0], 756)):

  bs = 54

  x_train = train_images['images'][image_index:image_index+756]
  y_train = train_images['labels'][image_index:image_index+756]

  ds = TensorDataset(x_train, y_train)
  dl = DataLoader(ds, batch_size = bs, shuffle=True)

  model, metrics = run_cnn(model, epochs, learning_rate, loss_func, opt, dl)
  metrics_dict['round_'+str(i)] = metrics
  
  f = open("/content/gdrive/My Drive/metrics_dict_train_4%s.pkl"%(str(i)),"wb")
  pickle.dump(metrics_dict,f)
  print('Finshed with set', str(i), 'of 504 images\nStarting next set.\n\n')

epoch 1
batch 1
Batch Loss tensor(0.7010, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.5926)
batch 2
Batch Loss tensor(0.7562, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.5926)
batch 3
Batch Loss tensor(0.5570, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.7407)
batch 4
Batch Loss tensor(0.9328, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.5556)
batch 5
Batch Loss tensor(0.6366, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.6667)
batch 6
Batch Loss tensor(0.5986, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.5926)
batch 7
Batch Loss tensor(0.6287, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.6667)
batch 8
Batch Loss tensor(0.4451, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.7593)
batch 9
Batch Loss tensor(0.6369, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.5926)
batch 10
Batch Loss tensor(0.4660, grad_fn=<NllLossBackward0>)
Batch Accuracy tensor(0.8333)
batch 11
Batch Loss tensor(0.6030, grad_fn=<NllLossBackward0>)
Batch Ac

## Training

In [20]:
# infile = open('/content/gdrive/My Drive/metrics_dict_train_20.pkl','rb')
# best_model2 = pickle.load(infile)

In [19]:
# for x in best_model2['round_0']['epoch_6']['accuracy']:
#   print(int(x))